### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [7]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_6


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    # model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 방대한 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 학습하는 사전 훈련(pre‑training) 과정을 거칩니다. '
 '이 과정에서 Transformer 구조와 자기‑주의(self‑attention) 메커니즘을 활용해 문맥을 이해하고 정보를 효율적으로 '
 '통합합니다. 이후 인간 피드백을 기반으로 한 강화 학습(RLHF) 등을 통해 모델을 미세 조정하여 보다 안전하고 유용한 응답을 생성하도록 '
 '최적화합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [9]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('ChatGPT는 인터넷의 방대한 글을 미리 학습해 두고, 사용자가 입력한 문장 다음에 올 법한 단어들의 확률을 계산해 문장을 만들어 '
 '냅니다.  \n'
 '학습 때는 “마스크”로 일부 단어를 가려 놓고 이를 맞추는 방식으로 언어 패턴을 익히며, 대화 맥락을 유지하기 위해 트랜스포머라는 구조를 '
 '반복 활용합니다.  \n'
 '이후 사람이 직접 대화 품질을 평가해 주는 “강화학습”을 거치면서 더 자연스럽고 유용한 답변을 만들어내도록 미세조정됩니다.\n'
 '\n'
 'ChatGPT 모델의 장점 요약\n'
 '• 문맥을 길게 기억해 자연스럽고 연결된 대화 가능  \n'
 '• 다양한 주제에 대해 높은 이해도와 논리적 답변 제공  \n'
 '• 코드 작성, 번역, 요약, 아이디어 생성 등 활용도가 뛰어남  \n'
 '• 사용자 의도에 따라 톤·스타일을 유연하게 조절 가능  \n'
 '• 지속적인 강화학습·업데이트로 성능이 빠르게 개선됨\n'
 '\n'
 'ChatGPT와 비슷한 AI 모델\n'
 '• 구글 제미나이(Gemini)  \n'
 '• 메타 라마(Llama)  \n'
 '• 클라우드 앤서로픽 클라우드(Claude)  \n'
 '• 카카오 브레인 코넥트(KoGPT)')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [10]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 방대한 텍스트 데이터를 바탕으로 단어·문맥 간 확률 관계를 학습하는 ‘Transformer’ 아키텍처를 이용합니다.  \n'
 '학습 시 입력 문장의 일부를 가리고 다음 토큰을 맞히는 ‘자기 지도 학습’ 방식으로 모델 내부 가중치를 반복 조정합니다.  \n'
 '이렇게 얻은 패턴과 지식을 바탕으로 새로운 입력에 대해 가장 가능성 높은 다음 단어를 순차적으로 생성합니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 대규모 텍스트·이미지·오디오 등 다중 모달 데이터를 동시에 학습해 패턴을 포착합니다.  \n'
 'Transformer 기반 아키텍처로, 입력 토큰 간 관계를 어텐션 메커니즘으로 계산하며 다음 토큰을 예측합니다.  \n'
 '강화학습과 인간 피드백(RLHF)으로 정답 경향을 반영해 보상을 최대화하며 정밀도를 높입니다.  \n'
 '전체 파라미터를 대상으로 분산 처리하며, 학습률 스케줄링과 정규화로 과적합을 억제해 확장 가능합니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('클로드는 대규모 언어 모델로, 인터넷·책 등 방대한 텍스트를 통해 다음 토큰이 무엇일지를 확률적으로 예측하는 방식으로 학습합니다.  \n'
 '학습 과정에서 트랜스포머 아키텍처의 어텐션 메커니즘이 문맥을 기억·활용해 단어 간 관계를 파악합니다.  \n'
 '지식이 눈덩이처럼 쌓이는 ‘스케일링’ 덕분에 모델 크기와 데이터가 커질수록 성능이 급격히 향상됩니다.  \n'
 '이후 인간 피드백 강화학습(RLHF)으로 도움·무해·정직 

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [11]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “대규모 언어 모델(Large Language Model, LLM)” 계열로, **‘다음 토큰 예측(next-token prediction)’**이라는 단순한 원리를 극단적으로 확장한 결과물입니다. 핵심 흐름은 (1) **사전학습(pre-training)** → (2) **지도미세조정(Supervised Fine-Tuning, SFT)** → (3) **강화학습 기반 인간 피드백 조정(RLHF)** 의 3단계로 설명할 수 있습니다.

---

### 1. 사전학습(Pre-training) : “인터넷 글을 암기하다”
- **목표**  
  주어진 앞 문장(컨텍스트)를 보고 다음에 올 법한 토큰(일반적으로 1~4 글자 분량)을 확률적으로 맞추는 것.

- **데이터**  
  공공 웹, 도서, 위키, 깃허브 등 수천억 토큰 규모의 “정제되지 않은” 대규모 코퍼스.

- **모델 구조**  
  Transformer 디코더(예: GPT-3 175B → 96층, 96 head, 12888 차원 임베딩).  
  셀프어텐션을 통해 임의 거리의 단어 관계를 계산.

- **학습 방식**  
  - 입력 시퀀스에 대해 각 위치의 정답 토큰에 대한 음의 로우 확률(negative log-likelihood)을 최소화  
  - 배치 크기 3.2 M토큰, 3000억 토큰을 1~2 epoch 정도

In [12]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 “만들어진 뒤엔 단순한 자동완성기”지만, 그 자동완성 능력을 얻기까지 3단계의 학습 절차를 거칩니다. 각 단계에서 사용되는 데이터, 목표, 최적화 기법이 다르며, 이를 이해하면 “왜 ChatGPT가 대화를 할 수 있는지”를 설명할 수 있습니다. 아래는 2024년 4월 기준으로 공개된 정보만을 정리한 설명입니다.

----------------------------------------
1. 단계 0: 사전학습(Pre-training)  
   목표 → “세상에 존재하는 텍스트를 그대로 압축”  
   데이터 → 공개 웹, 도서, 코드 등 수천 어휘 토큰(약 10 TB 급)  
   방법 → “다음 토큰 예측”을 수식화한 최대가능도 추정(Maximum Likelihood)  
   결과물 → GPT-베이스(base) 모델  
   핵심 포인트  
   - Transformer의 디코더만 쌓은 ‘언어 모델’이며, 프롬프트가 주어지면 확률적으로 토큰을 이어 씀.  
   - 지식이 들어 있지만 “사람이 원하는 방식”으로 답을 하지는 않음(홀루시네이션, 편향, 안전성 미확보).  
   - 이 시점엔 “대화” 데이터는 거의 없으므로, 사용자 질문에 “왜 그런지 설명해줘”라고 해도 질문 자체를 계속 이어 쓰기만 함.

----------------------------------------
2. 단계 1: 지도 파인튜닝(Supervised Fine-Tuning, SFT)  
   목표 → “대화 형식을 익히고, 지시사항을 따르는 습관 기르기”  
   데이터 → 대화·지시·응답 쌍 10~100k 수준(품질 높은 ‘교사 데이터’)  
   방법 → 동일한 최대가능도 학습, 다만 레이블=‘올바른 응답’인 지시응답 쌍  
   결과물 → SFT 모델(‘GPT-지시’라고도 불림)  
   핵심 포인트  
   - “프롬프트: A, 응답: B” 형태를 반복 학습해, 모델이 “지시를 끝내고 응답을 시작”하는 패턴을 익힘.  
   - 여전

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [13]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [14]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [15]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성을 “바위”와 “가스” 두 덩어리로 나눠 한 줄씩 정리하면 다음과 같습니다.

수성 · 금성 · 지구 · 화성  
땅덩어리(암행성)이고, 꼬리별처럼 얇은 대기만 씌워져 있다.

목성 · 토성 · 천왕성 · 해왕성  
거대 가스덩어리(목행성)로, 고리·위성·극광 등 장신구가 화려하다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [18]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [19]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 일어나는 대표적인 지구과학 현상으로 **태풍(열대저기압)**을 꼽을 수 있습니다. 한국을 비롯한 동아시아 지역에서는 8월 말부터 10월 초까지 태풍이 가장 많이 발생하고, 가을이 될수록 남쪽에서 북쪽으로 이동하면서 한반도에 영향을 미치는 경우가 많기 때문입니다. 다만 “가을에만” 일어나는 현상은 아니며, 다른 계절에도 나타날 수 있는 현상과 겹치는 부분이 있습니다.  

아래는 **가을에 특히 활발히 나타나는 주요 지구과학 현상 3가지**입니다.

| 현상 | 주요 특징 및 원인 | 가을에 두드러지는 이유 |
|------|-------------------|------------------------|
| **1. 태풍·열대저기압** | 바다 위에서 형성된 저기압이 강한 바람·폭우·해일을 동반함. 남서쪽의 온난·다습한 해수와 대기 불안정이 필요. | 가을이 되면 서쪽 태평양의 해수면 온도가 아직 높아 태풍이 강하게 발달하고, 제트기류가 남쪽으로 내려와 태풍 경로를 한반도·동아시아로 끌어당깁니다. |
| **2. 가을철 대기 역전·연무(안개)·연무 현상** | 지표면이 빠르게 냉각되면서 차가운 공기가 지표면에 머물고, 위쪽에 따뜻한 공층이 형성돼 대기 역전이 발생. 수증기가 응결해 안개·연무가 자주 나타남. | 가을은 일교차가 크게 늘고, 밤에 급격히 냉각되는 경우가 많아 대기 역전이 쉽게 형성됩니다. 특히 강원도·내륙 지방에서 산악 안개, 평야에서는 연무가 자주 보입니다. |
| **3. 가을 강수·전선 활동 강화** | 한반도는 대륙성 고기압(시베리아 고기압)과 남쪽의 남동아시아 저기압이 교차하면서 전선이 활발히 움직임. 이때 강수와 급격한 기온 변화가 일어남. | 가을에 시베리아 고기압이 남하하면서 차가운 북풍이 남쪽의 온난·다습한 공기와 만나 전선을 형성합니다. 전선이 통과할 때는 비·눈·우박이 동반되는 

In [27]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 가을

 가을에 발생하는 자연 현상:
1.  **성운**: 가을 밤에 밝은 별자리인 페가수스자리 근처에서 관찰할 수 있는 성운은 가을의 대표적인 천문 현상 중 하나입니다. 이 시기에 공기가 맑고 추운 날씨로 인해 밤하늘에서 더 많은 별을 볼 수 있습니다. 이 시기에는 성운이 선명하게 나타날 때가 많습니다.
2.  **수확월**: 지구가 태양을 중심으로 회전하면서 가을에는 밤과 낮의 길이가 달라지기 때문에 낮이 짧아지고 밤이 길어집니다. 이로 인해 기온이 더욱 낮아지고 추운 날씨가 시작됩니다. 이 시기에 농작물이 익고 수확할 수 있습니다.
3.  **가을 단풍**: 가을에는 일교차로 인해 밤과 낮의 기온이 크게 달라집니다. 이러한 기온 변화로 인해 나무의 잎이 색이 변하고 예쁜 단풍이 나타납니다. 붉은색, 노란색, 주황색 등 다양한 색깔의 단풍이 계절의 변화를 나타냅니다.


In [25]:
# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")

NameError: name 'season_name' is not defined

#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [21]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## **2024년 4월 5일 환율 분석**

### **1. 현재 환율: 1달러 = 1377.98원**

2024년 4월 5일, 1달러의 가치는 1377.98원으로 평가되고 있습니다. 이는 원화 가치의 약세를 의미합니다. 아래는 다양한 측면에서 이 환율에 대한 분석입니다.

### **2. 경제 지표 분석**

- **글로벌 경제 상황:** 최근 달러화 강세의 주된 요인 중 하나는 미국 경제의 견조한 성장세와 인플레이션 압력입니다. 미국의 높은 금리 수준이 투자자들에게 매력적인 수익을 제공하고 있어 달러에 대한 수요가 증가하고 있습니다.

- **한국 경제:** 한국 경제는 대외 무역 의존도가 높기 때문에 환율 변동에 민감합니다. 수출 중심의 경제 구조로 인해 약한 원화가 수출에 긍정적인 영향을 줄 수 있지만, 수입 물가의 상승으로 이어져 국내 인플레이션을 자극할 수 있습니다.

### **3. 정책적 요인**

- **한국은행:** 한국은행은 최근 금리 동결을 결정하는 등 고물가와 경기 둔화 우려 속에서 통화정책을 신중하게 운영하고 있습니다. 추가적인 금리 인상이나 인하 여부는 경제 상황에 따라 달라질 것입니다.

- **FED(연준):** 미국 연방준비제도(FED)는 인플레이션 목표 달성을 위해 금리를 조정하고 있습니다. 향후 금리 정책이 달러 가치에 영향을 미칠 것입니다.

### **4. 시장 전망**

- **단기 전망:** 단기적으로는 글로벌 경제 상황과 주요 경제국의 통화 정책에 따라 환율이 변동할 것입니다. 지정학적 리스크나 무역 관련 이슈도 환율에 영향을 줄 수 있습니다.

- **장기 전망:** 장기적으로는 한국 경제의 펀더멘털과 글로벌 경제의 흐름이 중요합니다. 원화 가치의 안정화 여부는 인플레이션 관리, 무역 수지, 외국인 투자 유치 등에 달려 있습니다.

### **5. 개인 및 기업에 미치는 영향**

- 